In [ ]:
%load_ext autoreload
%autoreload 2
import sys; sys.path.insert(0, '..')

### Virtual Distillation

This notebook provides an implementation of virtual distillation error mitigation, as described by: https://arxiv.org/pdf/2011.07064

Virtual distillation is an error mitigation which can leverage M copies of a state $\rho$ to surpress the error term. Virtual distillation describes the approximation of the error-free expectation value of an operator $O$ as:

$$<O>_{corrected} = \dfrac{Tr(O\rho^M)}{Tr(\rho^M)}$$

As described in the paper, we make use of the following equality:
$$Tr(O\rho^M) = Tr(O^{\textbf{i}}S^{(M)}\rho^{\otimes M})$$

This equation allows us to not calculate $\rho^M$, but instead use $M$ copies of $\rho$.
Hence we can implement the pseudocode as seen in algorithm 1 in the paper. Note that this notebook only provides an implementation for $M=2$

In [1]:
import cirq
import mitiq
import numpy as np
from mitiq.benchmarks import generate_rb_circuits

M = 2

Z = np.array([[1, 0], [0, -1]])
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
I = np.eye(2)
SWAP = np.array([[1, 0, 0, 0], 
                 [0, 0, 1, 0], 
                 [0, 1, 0, 0], 
                 [0, 0, 0, 1]])


## Bell state
The following code provides a bell state circuit as our $\rho$. However any N-qubit circuit works here. The circuits are in cirq package format

In [2]:
def bell_state():
    '''
    This function returns a circuit that prepares a Bell state in cirq circuit format.
    '''

    circuit = cirq.Circuit()
    qubits = cirq.LineQubit.range(3)
    circuit.append(cirq.H(qubits[0]))
    circuit.append(cirq.CNOT(qubits[0], qubits[1]))

    return circuit

## M copies of $\rho$
If no copies of $\rho$ are provided, the following function can be used.

In [3]:
def M_copies_of_rho(rho: cirq.Circuit, M: int=2):
    '''
    Given a circuit rho that acts on N qubits, this function returns a circuit that copies rho M times in parallel.
    This means the resulting circuit has N * M qubits.
    '''
    
    # if M <= 1:
    #     print("warning: M_copies_of_rho is not needed for M <= 1")
    #     return rho

    N = len(rho.all_qubits())

    circuit = cirq.Circuit()
    qubits = cirq.LineQubit.range(N*M)

    for i in range(M):
        circuit += rho.transform_qubits(lambda q: qubits[q.x + N*i])

    return circuit

In [4]:
# Test the M_copies_of_rho function
circuit = bell_state()
print(M_copies_of_rho(circuit, 3))

0: ───H───@───────────────────
          │
1: ───────X───────────────────

2: ───────────H───@───────────
                  │
3: ───────────────X───────────

4: ───────────────────H───@───
                          │
5: ───────────────────────X───


## Applying swaps
A copy in this context is the specific copy of $\rho$ out of the M copies in the entire circuit. \
As can be seen in the paper, we need to allow easy access to coupling qubit n of copy 1 with qubit n of copy 2 for any n $\in$ [1,N] \
This access is done by performing a series of SWAP operations such that this pattern results in the circuit where qubit n of copy 1 is stacked above qubit n of copy 2.\
The SWAPs are returned as a list of tuples which store the indices of the qubits that have to be swapped in order. 

In [5]:
# This algorithm only works for M = 2
def generate_swaps(l: list) -> list[tuple]:

    if len(l) % 2 != 0:
        raise ValueError("The list must have an even number of elements, since M=2")

    N = len(l) // 2

    if sorted(l) != list(range(0,2*N)):
        raise ValueError("The list must contain all the integers from 0 to 2*N-1")

    correct_list = []
    for i in range(N):
        correct_list.append(i)
        correct_list.append(i+N)

    swaps = []
    for index, value in enumerate(correct_list):
        if l[index] != value:
            l_index = l.index(value)
            l[index], l[l_index] = l[l_index], l[index]
            swaps.append((index, l_index))


    return swaps

# applies swaps to check if the generate swaps algorithm works
def apply_swaps(swaps_list: list[tuple], list_to_permute: list[int]) -> list[int]:

    permuted_list = list_to_permute.copy()
    for swap in swaps_list:
        permuted_list[swap[0]], permuted_list[swap[1]] = permuted_list[swap[1]], permuted_list[swap[0]]

    return permuted_list

In [6]:
# Testing the function
# [0,1,2,3,4,5] should map to [0,3,1,4,2,5]
# [0,1,2,3,4,5,6,7] should map to [0,4,1,5,2,6,3,7]
swaps_1 = generate_swaps([0,1,2,3])
swaps_2 = generate_swaps([0,1,2,3,4,5,6,7])

print(swaps_1)
print(apply_swaps(swaps_1, [0,1,2,3]))
print(apply_swaps(swaps_2, [0,1,2,3,4,5,6,7]))

[(1, 2)]
[0, 2, 1, 3]
[0, 4, 1, 5, 2, 6, 3, 7]


## B_i matrix generation
The B matrix as seen in the paper is defined by using the diagonalization of the $S^{(2)}_i$ operator and the observable $O^{(2)}$. The result of this code is not equal to the paper. Note how it says that some elements have freedom of phase. Thats why the B matrices can be different.

In [7]:
def commutator(mat1, mat2):
    
    comm = mat1 @ mat2 - mat2 @ mat1
    return np.allclose(mat1 @ mat2, mat2 @ mat1), comm

def dagger(mat):
    return np.conjugate(mat).T

# working (?) for 1 qubit systems with M = 2
# TODO test
# equations are taken directly from the paper

def generate_B_i(O, S_up2):

    O_up2 = 1/2 * (np.kron(O, I) + np.kron(I, O)) # eq (7)

    _, B_i = np.linalg.eig(S_up2)
    print(B_i)

    # swap columns of B_i to match the order of the eigenvectors of A
    # TODO i dont know why this is needed, and it seems like a random permutation
    # but i assume the eq 12 and eq 13 statements are correct 
    # B_i = B_i[:, [2, 0, 1, 3]]

    s2 = np.sqrt(2)/2
    B_i = np.array([
        [1, 0, 0, 0],
        [0, s2, s2, 0],
        [0, s2, -s2, 0],
        [0,0,0,1]
    ])

    # eq (9)
    if not commutator(O_up2, S_up2)[0]:
        print("Violates equation 9 (commutator)")
        return None


    print(B_i @ S_up2 @ dagger(B_i))
    # eq (12)
    if not np.allclose(
        B_i @ S_up2 @ dagger(B_i),
        1/2 * (np.eye(4) + np.kron(O,I) - np.kron(I,O) + np.kron(O, O) )):
        
        print("Violates equation 12")
        return None

    # eq (13)
    if not np.allclose(
        B_i @ O_up2 @ S_up2 @ dagger(B_i), 
                    O_up2):
        print("Violates equation 13")
        return None
    
    return B_i

print(generate_B_i(Z, SWAP))

[[ 0.          0.          1.          0.        ]
 [ 0.70710678  0.70710678  0.          0.        ]
 [ 0.70710678 -0.70710678  0.          0.        ]
 [ 0.          0.          0.          1.        ]]
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  4.26642159e-17  0.00000000e+00]
 [ 0.00000000e+00 -4.26642159e-17 -1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 1.          0.          0.          0.        ]
 [ 0.          0.70710678  0.70710678  0.        ]
 [ 0.          0.70710678 -0.70710678  0.        ]
 [ 0.          0.          0.          1.        ]]


In [8]:
### Testing generate_B_i
print(generate_B_i(X, SWAP))

[[ 0.          0.          1.          0.        ]
 [ 0.70710678  0.70710678  0.          0.        ]
 [ 0.70710678 -0.70710678  0.          0.        ]
 [ 0.          0.          0.          1.        ]]
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  4.26642159e-17  0.00000000e+00]
 [ 0.00000000e+00 -4.26642159e-17 -1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Violates equation 12
None


## The algorithm
With everything prepared, we can apply the algorithm. \
This example uses the bell state as $\rho$. As operator $O$, the pauli Z is chosen. \
Every operator has a $B_i$ unitary that has to be applied, as can be seen in the paper, we define $B_i$ for the pauli Z operator

In [9]:
from mitiqlocal.vd.vd import *

In [10]:
rho = bell_state()
# rho = cirq.Circuit()
# qubits = cirq.LineQubit.range(2)
# rho.append(cirq.X(qubits[0]))

# print(rho)
results = execute_with_vd(rho, M, 10000)
print(results)

[0.0006, 0.0006]


### Noise simulation
Here we run the circuit with simulated depolarizing noise for a one qubit circuit that is the identity, no operations are applied on this one qubit circuit

In [11]:
# This function is used to calculate the expectation value of the Z operator on a one qubit circuit
def execute(circuit):
    """Returns Tr[ρ Z] where ρ is the state prepared by the circuit
    with depolarizing noise."""
    
    # density matrix
    dm = cirq.DensityMatrixSimulator().simulate(circuit).final_density_matrix 
    
    print(dm)
    return dm[0, 0].real - dm[1, 1].real

In [12]:
# Identity circuit
rho = cirq.Circuit()
qubits = cirq.LineQubit.range(1)
rho.append(cirq.I(qubits[0])) # apply identity

noise_level = 0.1
noisy_rho = rho.copy().with_noise(cirq.depolarize(p=noise_level))

true_value = execute(rho) # Fault tolerant quantum computer
noisy_value = execute(noisy_rho) # Noisy quantum computer
vd_value = execute_with_vd(noisy_rho, 2, 1000)[0] # Noisy quantum computer + virtual distillation

print(f"True value: {true_value}")
print(f"Noisy value: {noisy_value}")
print(f"Virtual distillation value: {vd_value}")
print(f"Error w/o  virtual distillation: {abs((true_value - noisy_value) / true_value):.3f}")
print(f"Error w virtual distillation:    {abs((true_value - vd_value) / true_value):.3f}")

[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[0.9333334 +0.j 0.        +0.j]
 [0.        +0.j 0.06666666+0.j]]
True value: 1.0
Noisy value: 0.8666667342185974
Virtual distillation value: 0.9798206278026906
Error w/o  virtual distillation: 0.133
Error w virtual distillation:    0.020


## Randomized benchmarking circuits
### 1 qubit circuits
As an example, here we run the algorithm with the pauli Z observable on random 1 qubit benchmarking circuits. These circuits are equal in function to identity. Which means all expectation values will be 1. We see that the Virtual Distillation results of noisy circuits are more accurate. 

I tested here that VD breaks with noise level > 0.08 using the B matrix from the paper \
My B found with the generate_B_i function seems to improve on the noisy circuit until noise level > .35

In [13]:
# returns a random benchmarking qubit circuit equal to identity
def random_circuit(n_qubits: int, num_cliffords: int) -> list:
    return mitiq.benchmarks.randomized_benchmarking.generate_rb_circuits(n_qubits=n_qubits, num_cliffords=num_cliffords, trials=1, return_type=None, seed=None)

for i in range(10):
    rho = random_circuit(1, 50)[0]
    
    noise_level = 0.01 + 0.001 * i
    noisy_rho = rho.copy().with_noise(cirq.depolarize(p=noise_level))

    print(noise_level)

    true_value = execute(rho) # Fault tolerant quantum computer
    noisy_value = execute(noisy_rho) # Noisy quantum computer
    vd_value = execute_with_vd(noisy_rho, 2, 200)[0] # Noisy quantum computer + virtual distillation

    print(f"circuit: {i}, noise level: {noise_level}")
    print(f"True value: {true_value}")
    print(f"Noisy value: {noisy_value}")
    print(f"Virtual distillation value: {vd_value}")
    print()


0.01
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[0.6237932+0.j 0.       +0.j]
 [0.       +0.j 0.3762064+0.j]]
circuit: 0, noise level: 0.01
True value: 1.0
Noisy value: 0.24758678674697876
Virtual distillation value: 0.5471698113207547

0.011
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[0.5998903+0.j 0.       +0.j]
 [0.       +0.j 0.4001043+0.j]]
circuit: 1, noise level: 0.011
True value: 1.0
Noisy value: 0.199785977602005
Virtual distillation value: 0.375

0.012
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[0.58480716+0.j 0.        +0.j]
 [0.        +0.j 0.4151933 +0.j]]
circuit: 2, noise level: 0.012
True value: 1.0
Noisy value: 0.16961386799812317
Virtual distillation value: 0.3137254901960784

0.013000000000000001
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[0.575653  +0.j 0.        +0.j]
 [0.        +0.j 0.42434204+0.j]]
circuit: 3, noise level: 0.013000000000000001
True value: 1.0
Noisy value: 0.1513109803199768
Virtual distillation value: 0.22413793103448276

0.014
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[0.561748

### 2 qubit circuits
here we see VD is not more accurate when noise level > 0.6 using the B from the paper \
Using the B from generate_B_i, when the noise level > 0.4 sometimes it does perform better other times it doesnt

In [14]:
# 2 qubit benchmarking circuits
for i in range(10):
    rho = random_circuit(2, 10)[0]
    
    noise_level = 0.01 * i
    noisy_rho = rho.copy().with_noise(cirq.depolarize(p=noise_level))

    true_value = execute(rho) # Fault tolerant quantum computer
    noisy_value = execute(noisy_rho) # Noisy quantum computer
    vd_value = execute_with_vd(noisy_rho, 2, 1000)[0] # Noisy quantum computer + virtual distillation

    print(f"circuit: {i}, noise level: {noise_level}")
    print(f"True value: {true_value}")
    print(f"Noisy value: {noisy_value}")
    print(f"Virtual distillation value: {vd_value}")
    print()

[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
circuit: 0, noise level: 0.0
True value: 1.0
Noisy value: 1.0
Virtual distillation value: 1.0

[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
[[0.41540003+0.j 0.        +0.j 0.        +0.j 0.        +0.j]
 [0.        +0.j 0.2003567 +0.j 0.        +0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.19139934+0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.        +0.j 0.19284262+0.j]]
circuit: 1, noise level: 0.01
True value: 1.0
Noisy value: 0.2150433212518692
Virtual distillation value: 0.46357615894039733

[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
[[0.28222632+0.j 0.      

KeyboardInterrupt: 

In [15]:
def add_noise(circuit, noise_level):
    """Apply depolarizing noise to a circuit."""
    return circuit.with_noise(cirq.depolarize(p=noise_level))

def identity_test(noise_levels, repetitions):
    """Run identity circuit tests with varying noise levels."""
    qubits = cirq.LineQubit.range(1)
    results = []

    for noise_level in noise_levels:
        circuit = cirq.Circuit(cirq.I(qubits[0]))
        noisy_circuit = add_noise(circuit, noise_level)

        true_value = execute(circuit)
        noisy_value = execute(noisy_circuit)
        vd_value = execute_with_vd(noisy_circuit, 2, repetitions)[0]

        results.append({
            "noise_level": noise_level,
            "true_value": true_value,
            "noisy_value": noisy_value,
            "vd_value": vd_value
        })
    return results

In [16]:
def randomized_benchmarking_test(n_qubits, num_cliffords, noise_levels, repetitions):
    """Run randomized benchmarking tests with varying noise levels."""
    results = []

    for noise_level in noise_levels:
        circuit = generate_rb_circuits(n_qubits=n_qubits, num_cliffords=num_cliffords, trials=1)[0]
        noisy_circuit = add_noise(circuit, noise_level)

        true_value = execute(circuit)
        noisy_value = execute(noisy_circuit)
        vd_value = execute_with_vd(noisy_circuit, 2, repetitions)[0]

        results.append({
            "noise_level": noise_level,
            "true_value": true_value,
            "noisy_value": noisy_value,
            "vd_value": vd_value
        })
    return results

def heisenberg_quench_test(n_qubits, time_steps, noise_levels, repetitions):
    """Run Heisenberg quench tests with varying noise levels."""
    results = []

    for noise_level in noise_levels:
        qubits = cirq.LineQubit.range(n_qubits)
        circuit = cirq.Circuit()

        for t in range(time_steps):
            for i in range(n_qubits - 1):
                circuit.append(cirq.CZ(qubits[i], qubits[i + 1]))
            for q in qubits:
                circuit.append(cirq.X(q)**0.5)

        noisy_circuit = add_noise(circuit, noise_level)

        true_value = execute(circuit)
        noisy_value = execute(noisy_circuit)
        vd_value = execute_with_vd(noisy_circuit, 2, repetitions)[0]

        results.append({
            "noise_level": noise_level,
            "true_value": true_value,
            "noisy_value": noisy_value,
            "vd_value": vd_value
        })
    return results

In [17]:
def print_results(results, test_name):
    """Print results of a test."""
    print(f"Results for {test_name}:")
    for i, result in enumerate(results):
        print(f"Trial {i}, Noise Level: {result['noise_level']}")
        print(f"True Value: {result['true_value']}")
        print(f"Noisy Value: {result['noisy_value']}")
        print(f"Virtual Distillation Value: {result['vd_value']}\n")

In [18]:
# Parameters
noise_levels = [0.01 * i for i in range(10)]
repetitions = 1000

# Identity Test
identity_results = identity_test(noise_levels, repetitions)
print_results(identity_results, "Identity Test")

# Randomized Benchmarking Test
rb_results = randomized_benchmarking_test(n_qubits=1, num_cliffords=50, noise_levels=noise_levels, repetitions=repetitions)
print_results(rb_results, "Randomized Benchmarking Test")

# Heisenberg Quench Test
heisenberg_results = heisenberg_quench_test(n_qubits=3, time_steps=5, noise_levels=noise_levels, repetitions=repetitions)
print_results(heisenberg_results, "Heisenberg Quench Test")


[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[0.99333334+0.j 0.        +0.j]
 [0.        +0.j 0.00666667+0.j]]
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[0.9866666 +0.j 0.        +0.j]
 [0.        +0.j 0.01333333+0.j]]
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[0.97999996+0.j 0.        +0.j]
 [0.        +0.j 0.02      +0.j]]
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[0.9733333 +0.j 0.        +0.j]
 [0.        +0.j 0.02666667+0.j]]
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[0.9666666 +0.j 0.        +0.j]
 [0.        +0.j 0.03333333+0.j]]
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[0.9599999+0.j 0.       +0.j]
 [0.       +0.j 0.04     +0.j]]
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[0.9533333 +0.j 0.        +0.j]
 [0.        +0.j 0.04666666+0.j]]
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[0.9466666 +0.j 0.        +0.j]
 [0.        +0.j 0.05333334+0.j]]
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[0.93999994+0.j 0.        +0.j]
 [0.        +0.j 0.06      +0.j]]
Results for Iden